In [ ]:
!pip install transformers

In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl

In [ ]:
# !git lfs install
# !git clone https://huggingface.co/nlpaueb/legal-bert-base-uncased

In [ ]:
import pandas as pd
import os
import numpy as np
import re
import os
import torch
from torch.cuda import amp
import torch.nn as nn
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import TensorDataset, Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW
import time
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import matthews_corrcoef
from tqdm import tqdm
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

ModuleNotFoundError: ignored

In [ ]:
!git lfs install

!git clone https://huggingface.co/nlpaueb/bert-base-uncased-eurlex
# !git clone https://huggingface.co/nlpaueb/legal-bert-base-uncased

# if you want to clone without large files – just their pointers
# prepend your git clone with the following env var:
GIT_LFS_SKIP_SMUDGE=1 

git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	log
fatal: destination path 'bert-base-uncased-eurlex' already exists and is not an empty directory.


In [ ]:
device = xm.xla_device()
# specify GPU
# if torch.cuda.is_available():    

#     # Tell PyTorch to use the GPU.    
#     device = torch.device("cuda")

#     print('There are %d GPU(s) available.' % torch.cuda.device_count())

#     print('We will use the GPU:', torch.cuda.get_device_name(0))

# # If not...
# else:
#     print('No GPU available, using the CPU instead.')
#     device = torch.device("cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_cite = pd.read_excel('drive/MyDrive/Data/Euthority/df_cite.xlsx')
df_cite.rename(columns = {'CELEX_ID': 'celex_num'}, inplace=True)


In [ ]:
dff = pd.read_excel('drive/MyDrive/Data/Euthority/eng_leg_text.xlsx', index_col=0)
print(dff.columns)
dff['text_size'] = dff.text.apply(lambda x : len(x.split()))
# print(df.head())

df = pd.merge(dff, df_cite, on='celex_num', how='inner')
df['year'] = pd.to_datetime(df.celex_num.apply(lambda x : x[1:5])).dt.year

Index(['celex_num', 'text', 'text_size', 'descriptor', 'short_text',
       'language'],
      dtype='object')


In [ ]:
df0 = df[(df['cited'] == 0) & (df['text_size'] > 200)].sample(frac=0.5, random_state=2018)
df1 = df[df['cited'] == 1]
print(df0.shape, df1.shape)
print(df1.text_size.describe())
print(df0.text_size.describe())

(36008, 8) (5170, 8)
count    5170.000000
mean     1095.793617
std      1428.502146
min         3.000000
25%       245.000000
50%       491.500000
75%      1181.500000
max      6427.000000
Name: text_size, dtype: float64
count    36008.000000
mean       647.568957
std        894.719110
min        201.000000
25%        258.000000
50%        366.000000
75%        582.000000
max       6431.000000
Name: text_size, dtype: float64


In [ ]:
df1_tr = pd.DataFrame()
df1_te = pd.DataFrame()
for i in df1.year.unique():
  a = df1[df1['year'] == i]
  b = a.sample(frac=0.7, random_state=2018)
  c = a[~a.celex_num.isin(b.celex_num)]
  df1_tr = df1_tr.append(b)
  df1_te = df1_te.append(c)
print(df1_tr.shape, df1_te.shape)

df1_common = pd.merge(df1_tr, df1_te, on='celex_num', how='inner')
print(df1_common.shape)
del df1_common

df0_tr, df0_te = pd.DataFrame(), pd.DataFrame()
for i in df0.year.unique():
  a = df0[df0['year'] == i]
  b = a.sample(frac=0.7, random_state=2018)
  c = a[~a.celex_num.isin(b.celex_num)]
  df0_tr = df0_tr.append(b)
  df0_te = df0_te.append(c)
print(df0_tr.shape, df0_te.shape)


df0_common = pd.merge(df0_tr, df0_te, on='celex_num', how='inner')
print(df0_common.shape)
del df0_common


(3623, 8) (1547, 8)
(0, 15)
(25205, 8) (10803, 8)
(0, 15)


In [ ]:
# train_set
# oversampling the imbalanced class in the train set but not in the test set
df_train = pd.DataFrame()
df_train = df_train.append(df0_tr)
for i in range(3):
  df_train = df_train.append(df1_tr)

df_train = df_train.sample(frac=1, random_state=2018)
del df0_tr, df1_tr
# test set
df_test = pd.DataFrame()
df_test = df_test.append(df0_te)
df_test = df_test.append(df1_te)
df_test = df_test.sample(frac=1, random_state = 2018)
del df0_te, df1_te

df_train.reset_index(inplace=True)
df_test.reset_index(inplace=True)
print(df_train.shape, df_test.shape)

(32451, 9) (12350, 9)


In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# bert = BertModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/bert-base-uncased-eurlex", do_lower_case=True)

model = AutoModel.from_pretrained("nlpaueb/bert-base-uncased-eurlex")

In [ ]:
class BERTDataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
    self.data = dataframe
    self.text = self.data.text
    self.targets = self.data.cited
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.text)

  def __getitem__(self, idx):
    text = str(self.text[idx])
    text = " ".join(text.split())

    inputs = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        pad_to_max_length=True,
        truncation = True,
        max_length = self.max_len,
        return_token_type_ids=False
    )

    ids = inputs['input_ids']
    mask = inputs['attention_mask']
    # token_type_ids = inputs['token_type_ids']

    return {
        'input_ids': torch.tensor(ids, dtype=torch.long, device=device),
        'attention_mask': torch.tensor(mask, dtype=torch.long, device=device),
        # 'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long, device=device),
        'label': torch.tensor(self.targets[idx], dtype=torch.long, device=device)
    }

NameError: ignored

In [ ]:
max_len=480
train_dataset = BERTDataset(df_train, tokenizer, max_len)
test_dataset = BERTDataset(df_test, tokenizer, max_len)

In [ ]:
train_batch_size, test_batch_size = 8,4
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size = train_batch_size, drop_last=True)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size = test_batch_size)

In [ ]:
# for param in model.parameters():
#   param.requires_grad = False

class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # # dense layer 1
      self.fc1 = nn.Linear(768,2)
      
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,2)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):
      # print(sent_id)
      #pass the inputs to the model  
      out = self.bert(sent_id, mask)
      # print(cls_hs)
      x = self.fc1(out.pooler_output)

      x = self.dropout(x)

      # output layer
      x = self.fc2(x)
      
      # apply softmax activation
      x = self.softmax(x)
      
      y = out.pooler_output
      
      return x, y

In [ ]:
# pass the pre-trained BERT to our define architecture
bert_model = BERT_Arch(model)

# push the model to GPU
bert_model = bert_model.to(device)


In [ ]:
# define the optimizer
optimizer = AdamW(bert_model.parameters(), lr = 3e-4)

from sklearn.utils.class_weight import compute_class_weight

# #compute the class weights
class_wts = compute_class_weight('balanced', np.unique(df_train.cited.tolist()), df_train.cited.tolist())

print(class_wts)
# convert class weights to tensor
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)



[0.64374132 2.23923544]


In [ ]:
# loss function
def loss_fn(outputs, targets, weights):
    # print('outputs: ', outputs, outputs.size())
    # print('targets: ', targets, targets.size())
    # return nn.BCEWithLogitsLoss()(outputs, targets.view(-1, 1))    
    cross_entropy = nn.NLLLoss(weight=weights)
    return cross_entropy(outputs, targets)
# number of training epochs 


In [ ]:
def train_loop_fn(dataloader, model, optimizer, device, weights):
    model.train()
    total_loss = 0
    
    # empty list to save model predictions
    total_preds, train_targets=[], []
 
    for step, batch in enumerate(dataloader):
        # print(len(batch))
        # progress update after every 50 batches.
        if step % 500 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))
        
        # push the batch to gpu
        batch = [batch[str(r)].to(device) for r in batch]
        input_id, mask, targets = batch
      
        optimizer.zero_grad()
        # with amp.autocast():
        out_x, out_y = model(input_id, mask)
        
        loss = loss_fn(out_x, targets, weights)


        total_loss += loss.item()
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        xm.optimizer_step(optimizer, barrier=True)
        
        
        # model predictions are stored on GPU. So, push it to CPU
        total_preds.extend(out_y.detach().cpu().numpy())
        train_targets.extend(targets.detach().cpu().numpy())

    # compute the training loss of the epoch
    # avg_loss = total_loss / len(dataloader)

    # predictions are in the form of (no. of batches, size of batch, no. of classes).
    # reshape the predictions in form of (number of samples, no. of classes)
    # total_preds  = np.concatenate(total_preds, axis=0)
    xgb_df_train = pd.DataFrame(total_preds)
    
    #returns the loss and predictions
    return xgb_df_train, train_targets

In [ ]:
def eval_fn(eval_dataloader, model, device):
    model.eval()
    preds, true_labels = [], []
    # with torch.no_grad():
    for step, batch in enumerate(eval_dataloader):

        # push the batch to gpu
        batch = [batch[str(r)].to(device) for r in batch]
        input_id, mask, targets = batch

        out_x, out_y = model(input_id, mask)

        true_labels.extend(targets.detach().cpu().numpy())
        preds.extend(out_y.detach().cpu().numpy())

    # reshape the predictions in form of (number of samples, no. of classes)
    # preds  = np.concatenate(preds, axis=0)
    xgb_df_test = pd.DataFrame(preds)

    return xgb_df_test, true_labels

In [ ]:
epochs = 3
# # num_training_steps = int(len(df_train) / (batch_size * epochs))
# # print(num_training_steps)
# # scaler = amp.GradScaler()
for epoch in range(epochs):
    train_df, train_target = train_loop_fn(train_dataloader, bert_model, optimizer, device, weights)


In [ ]:
# print(train_df.head())

In [ ]:
# test_df, test_target = eval_fn(test_dataloader, bert_model, device)
# print(test_df.shape)
# print(test_target.shape)

In [ ]:
# xgb_train_df = pd.DataFrame(train_list)
# print(xgb_train_df.shape)
# xgb_train_df = train_df.copy()
# xgb_train_df['label'] = 

In [ ]:
# import xgboost as xgb

In [ ]:
# preparing dmatrix for xgboost
# d_train = xgb.DMatrix(train_df, label = train_target)
# d_test = xgb.DMatrix(test_df, label = test_target)

# extreme gradient boosting
# model_name = 'XGBoost'
# parameters = {"max_depth":15, "eta":0.005, "gamma":1, "sampling_method":"uniform", \
#               "subsample":0.5, "scale_pos_weight":4, "objective":"binary:logistic", \
#               "verbosity":0}

In [ ]:
# model = xgb.train(parameters, d_train, 50)

In [ ]:
# test_pred = model.predict(d_test)
# print(test_pred)
# y_pred = np.where(test_pred <= 0.5, 0, 1)

Increase oversampling and check performance \
Try XGBoost after the pooler output \
Try better hyper parameters \
Try the other loss functions for binary classification


In [ ]:

# print('F1-score: ', f1_score(test_target, y_pred))
# print(classification_report(test_target, y_pred))